In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import pickle
import gc
import time
import itertools

from tqdm import tqdm
from sklearn.metrics import roc_auc_score, auc, roc_auc_score
from contextlib import contextmanager
from pathlib import Path
from time import time 
from collections import namedtuple
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
gc.enable()

import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn_utils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import joblib
import random

In [ ]:
%%time
skills = joblib.load("../input/data-generate/skills.pkl.zip")
n_skill = len(skills)
group = pd.read_pickle("../input/group-with-label-smothing/group_with_labelsmoothing.pkl")

In [ ]:
group[115]

In [ ]:
'''
%%time
n=len(group.index)
VALID_SAMPLES = 0
valid_indexes=np.random.choice(group.index, VALID_SAMPLES, replace=False)
train_group = group[np.invert(group.index.isin(valid_indexes))]
valid_group =group[group.index.isin(valid_indexes)]
'''

In [ ]:
MAX_SEQ=100
class RiiidDataset(Dataset):
    def __init__(self,group,n_skill,max_seq=MAX_SEQ):
        super(RiiidDataset,self).__init__()
        self.max_seq = max_seq
        self.n_skill = n_skill
        self.samples = {}
        self.user_ids = []
        for user_id in group.index:
            _,content_id_,answered_correctly_,part_id_, \
                prior_question_elapsed_time_,prior_lag_time_,\
            answered_correctly_smoothing_= group[user_id]
            if len(content_id_)<5:
                continue
            if len(content_id_)>self.max_seq:
                total_questions = len(content_id_)
                initial = total_questions % self.max_seq
                if initial >= 5:
                    self.user_ids.append(f"{user_id}_0")
                    self.samples[f"{user_id}_0"] = (content_id_[:initial],
                                    answered_correctly_[:initial],part_id_[:initial],
                                        prior_question_elapsed_time_[:initial],
                                        prior_lag_time_[:initial],answered_correctly_smoothing_[:initial])
                    
                for seq in range(total_questions // self.max_seq):
                    self.user_ids.append(f"{user_id}_{seq+1}")
                    start = initial + seq * self.max_seq
                    end = initial + (seq + 1) * self.max_seq
                    self.samples[f"{user_id}_{seq+1}"] = (content_id_[start:end],
                    answered_correctly_[start:end],part_id_[start:end], 
                        prior_question_elapsed_time_[start:end],prior_lag_time_[start:end],
                       answered_correctly_smoothing_[start:end]  )
                '''
                initial = total_questions % self.max_seq
                a=random.random()
                if a>=0.75 and initial >= 50:
                    
                    self.user_ids.append(f"{user_id}_0_50")
                    self.samples[f"{user_id}_0_50"] = (content_id_[:initial],
                                    answered_correctly_[:initial],part_id_[:initial],
                                    prior_question_elapsed_time_[:initial],
                                   prior_lag_time_[:initial],answered_correctly_smoothing_[:initial]) 
                    for seq in range(total_questions // self.max_seq):
                        self.user_ids.append(f"{user_id}_{seq+1}_50")
                        start = initial + seq * self.max_seq
                        end = initial + (seq + 1) * self.max_seq
                        self.samples[f"{user_id}_{seq+1}_50"] = (content_id_[start:end],
                        answered_correctly_[start:end],part_id_[start:end], 
                            prior_question_elapsed_time_[start:end],prior_lag_time_[start:end],
                           answered_correctly_smoothing_[start:end]  )

                if len(content_id_)>80:
                    n_sample = total_questions // self.max_seq
                    for i in range(n_sample):
                        index=np.random.randint(0,len(content_id_)-1,150)
                        index=sorted(list(set(index)))[:100]
                        self.user_ids.append(f"{user_id}_{seq+1}"+str(i))
                        self.samples[f"{user_id}_{seq+1}"+str(i)] = (content_id_[index],
                                    answered_correctly_[index],part_id_[index],
                                        prior_question_elapsed_time_[index],
                                        prior_lag_time_[index],answered_correctly_smoothing_[index])  
                    '''
            else:
                user_id = str(user_id)
                self.user_ids.append(user_id)
                self.samples[user_id] = (content_id_,answered_correctly_,
                                         part_id_,prior_question_elapsed_time_,
                                       prior_lag_time_,answered_correctly_smoothing_)

    def __len__(self):
        return len(self.user_ids)
    
    def __getitem__(self,index):
        user_id = self.user_ids[index]
        content_id_,answered_correctly_,\
        part_id_,prior_question_elapsed_time_,\
        prior_lag_time_,answered_correctly_smoothing_= self.samples[user_id]
        
        seq_len = len(content_id_)
        
        content_id=np.zeros(self.max_seq,dtype = int)
        answered_correctly=np.zeros(self.max_seq,dtype = int)
        part_id=np.zeros(self.max_seq,dtype = int)
        prior_question_elapsed_time=np.zeros(self.max_seq,dtype = int)
        prior_lag_time=np.zeros(self.max_seq,dtype = int)
        
        answered_correctly_smoothing=np.zeros(self.max_seq,dtype = float)
        
        if seq_len == self.max_seq:
            content_id[:] = content_id_
            answered_correctly[:] = answered_correctly_
            part_id[:] = part_id_
            prior_question_elapsed_time[:] = prior_question_elapsed_time_
            prior_lag_time[:] = prior_lag_time_
            answered_correctly_smoothing[:] =  answered_correctly_smoothing_

        else:
            content_id[-seq_len:] = content_id_
            answered_correctly[-seq_len:] = answered_correctly_
            part_id[-seq_len:] = part_id_
            prior_question_elapsed_time[-seq_len:] = prior_question_elapsed_time_
            prior_lag_time[-seq_len:] = prior_lag_time_
            answered_correctly_smoothing[-seq_len:] =  answered_correctly_smoothing_


        content_id_new=content_id[1:]
        answered_correctly_new=answered_correctly[1:]
        part_id_new=part_id[1:]
        prior_question_elapsed_time_new=prior_question_elapsed_time[1:]
        prior_lag_time_new = prior_lag_time[1:]
        answered_correctly_smoothing_new=answered_correctly_smoothing[1:]
        
        x = np.zeros(self.max_seq-1, dtype=int)
        x = content_id[:-1].copy()
        x += (answered_correctly[:-1] == 1) * self.n_skill
 
        return x, content_id_new,part_id_new,\
prior_question_elapsed_time_new,prior_lag_time_new,answered_correctly_new,\
answered_correctly_smoothing_new


    
def collate_fn(batch):
    x, content_id,part_id, prior_question_elapsed_time,\
    prior_lag_time,label,label_smoothing = zip(*batch)
    
    x=torch.Tensor(x).long()
    content_id = torch.Tensor(content_id).long()
    part_id = torch.Tensor(part_id).long()
    prior_question_elapsed_time = torch.Tensor(prior_question_elapsed_time).long()
    prior_lag_time = torch.Tensor(prior_lag_time).long()
    label = torch.Tensor(label).float()
    label_smoothing = torch.Tensor(label_smoothing).float()
    # remember the order
    return x,content_id,part_id, prior_question_elapsed_time,\
            prior_lag_time,label,label_smoothing

In [ ]:
class ScaledDotProductAttention(nn.Module):
    
    def __init__(self, temperature, attn_dropout=0.1):
        super().__init__()
        self.temperature = temperature
        self.dropout = nn.Dropout(attn_dropout)
    
    def forward(self, q, k, v, mask=None):
        attn = torch.matmul(q / self.temperature, k.transpose(2, 3))
        
        if mask is not None:
            attn = attn.masked_fill(mask == 0, -1e9)
            
        attn = self.dropout(F.softmax(attn, dim=-1))
        output = torch.matmul(attn, v)
        
        return output, attn
class MultiHeadAttention(nn.Module):
    def __init__(self,n_head,d_model,d_k,d_v,dropout=0.1):
        super().__init__()
        
        self.n_head = n_head
        self.d_k = d_k
        self.d_v = d_v

        self.w_qs = nn.Linear(d_model, n_head * d_k, bias=False)
        self.w_ks = nn.Linear(d_model, n_head * d_k, bias=False)
        self.w_vs = nn.Linear(d_model, n_head * d_v, bias=False)
        self.fc = nn.Linear(n_head * d_v, d_model, bias=False)

        self.attention = ScaledDotProductAttention(temperature=d_k ** 0.5)

        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)
    
    def forward(self, q, k, v, mask=None):
        
        d_k, d_v, n_head = self.d_k, self.d_v, self.n_head
        sz_b, len_q, len_k, len_v = q.size(0), q.size(1), k.size(1), v.size(1)
        
        residual = q
        
        q = self.w_qs(q).view(sz_b, len_q, n_head, d_k)
        k = self.w_ks(k).view(sz_b, len_k, n_head, d_k)
        v = self.w_vs(v).view(sz_b, len_v, n_head, d_v)
        
        q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2)
        
        if mask is not None:
            mask = mask.unsqueeze(1)   # For head axis broadcasting.
        
        q, attn = self.attention(q, k, v, mask=mask)

        # Transpose to move the head dimension back: b x lq x n x dv
        # Combine the last two dimensions to concatenate all the heads together: b x lq x (n*dv)
        q = q.transpose(1, 2).contiguous().view(sz_b, len_q, -1)
        q = self.dropout(self.fc(q))
        q += residual

        q = self.layer_norm(q)

        return q, attn
class PositionwiseFeedForward(nn.Module):
    ''' A two-feed-forward-layer module '''

    def __init__(self, d_in, d_hid, dropout=0.2):
        super().__init__()
        self.w_1 = nn.Linear(d_in, d_hid) # position-wise
        self.w_2 = nn.Linear(d_hid, d_in) # position-wise
        #self.layer_norm = nn.LayerNorm(d_in, eps=1e-6)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):

        residual = x

        x = self.w_2(F.relu(self.w_1(x)))
        x = self.dropout(x)
        x += residual
        #x = self.layer_norm(x)

        return x
class EncoderLayer(nn.Module):
    ''' Compose with two layers '''

    def __init__(self, d_model, d_inner, n_head, d_k, d_v, dropout=0.1):
        super(EncoderLayer, self).__init__()
        self.slf_attn = MultiHeadAttention(n_head, d_model, d_k, d_v, dropout=dropout)
        self.pos_ffn = PositionwiseFeedForward(d_model, d_inner, dropout=dropout)

    def forward(self, enc_input, slf_attn_mask=None):
        enc_output, enc_slf_attn = self.slf_attn(
            enc_input, enc_input, enc_input, mask=slf_attn_mask)
        enc_output = self.pos_ffn(enc_output)
        return enc_output, enc_slf_attn

class DecoderLayer(nn.Module):

    def __init__(self, d_model, d_inner, n_head, d_k, d_v, dropout=0.1):
        super(DecoderLayer, self).__init__()
        self.slf_attn = MultiHeadAttention(n_head, d_model, d_k, d_v, dropout=dropout)
        self.enc_attn = MultiHeadAttention(n_head, d_model, d_k, d_v, dropout=dropout)
        self.pos_ffn = PositionwiseFeedForward(d_model, d_inner, dropout=dropout)

    def forward(
            self, dec_input, enc_output,
            slf_attn_mask=None, dec_enc_attn_mask=None):
        dec_output, dec_slf_attn = self.slf_attn(
            dec_input, dec_input, dec_input, mask=slf_attn_mask)
        dec_output, dec_enc_attn = self.enc_attn(
            dec_output, enc_output, enc_output, mask=dec_enc_attn_mask)
        dec_output = self.pos_ffn(dec_output)
        return dec_output, dec_slf_attn, dec_enc_attn

class PositionalEncoding(nn.Module):
    def __init__(self, d_hid, n_position=100):
        super(PositionalEncoding, self).__init__()

        # Not a parameter
        self.register_buffer('pos_table', self._get_sinusoid_encoding_table(n_position, d_hid))

    def _get_sinusoid_encoding_table(self, n_position, d_hid):
        ''' Sinusoid position encoding table '''
        # TODO: make it with torch instead of numpy

        def get_position_angle_vec(position):
            return [position / np.power(10000, 2 * (hid_j // 2) / d_hid) for hid_j in range(d_hid)]

        sinusoid_table = np.array([get_position_angle_vec(pos_i) for pos_i in range(n_position)])
        sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i
        sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1

        return torch.FloatTensor(sinusoid_table).unsqueeze(0)

    def forward(self, x,y):
        return x + self.pos_table[:, 1:x.size(1)+1].clone().detach(),\
    y + self.pos_table[:, :x.size(1)].clone().detach()

def future_mask(seq_length):
    return (1 - torch.triu(torch.ones((1, seq_length, seq_length)), diagonal=1)).bool()

class Encoder(nn.Module):
    ''' A encoder model with self attention mechanism. '''

    def __init__(
            self,d_model,n_layers, n_head, d_k, d_v, d_inner, dropout=0.1):

        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.layer_stack = nn.ModuleList([
            EncoderLayer(d_model, d_inner, n_head, d_k, d_v, dropout=dropout)
            for _ in range(n_layers)])
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)

    def forward(self, x,  return_attns=False):

        enc_slf_attn_list = []  
        enc_output = self.dropout(x)
        enc_output = self.layer_norm(enc_output)
        att_mask = future_mask(x.size(1)).to(device)

        for enc_layer in self.layer_stack:
            enc_output, enc_slf_attn = enc_layer(enc_output, slf_attn_mask=att_mask)
            enc_slf_attn_list += [enc_slf_attn] if return_attns else []

        if return_attns:
            return enc_output, enc_slf_attn_list
        return enc_output,_

class Decoder(nn.Module):
    ''' A decoder model with self attention mechanism. '''

    def __init__(
            self,  d_model, n_layers, n_head, d_k, d_v,
            d_inner,dropout=0.1):

        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        self.layer_stack = nn.ModuleList([
            DecoderLayer(d_model, d_inner, n_head, d_k, d_v, dropout=dropout)
            for _ in range(n_layers)])
        self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)

    def forward(self, x, enc_output,  return_attns=False):

        dec_slf_attn_list, dec_enc_attn_list = [], []
        
        att_mask = future_mask(x.size(1)).to(device)

        dec_output = self.dropout(x)
        dec_output = self.layer_norm(dec_output)

        for dec_layer in self.layer_stack:
            dec_output, dec_slf_attn, dec_enc_attn = dec_layer(
                dec_output, enc_output, slf_attn_mask=att_mask, dec_enc_attn_mask=att_mask)
            dec_slf_attn_list += [dec_slf_attn] if return_attns else []
            dec_enc_attn_list += [dec_enc_attn] if return_attns else []

        if return_attns:
            return dec_output, dec_slf_attn_list, dec_enc_attn_list
        return dec_output,_,_

In [ ]:
class SAINTPLUS(nn.Module):
    def __init__(self, n_skill, max_seq=MAX_SEQ, embed_dim=128, d_inner=128,
            n_layers=2, n_head=8, d_k=64, d_v=64, dropout=0.1, n_position=100): 
        super(SAINTPLUS, self).__init__()
        self.n_skill = n_skill
        self.embed_dim = embed_dim

        self.embedding = nn.Embedding(2*n_skill+1, embed_dim)
        self.pos_embedding = PositionalEncoding(embed_dim, n_position=100)
        self.e_embedding = nn.Embedding(n_skill+1, embed_dim)
        self.p_embedding = nn.Embedding(8, embed_dim)
        self.t_embedding = nn.Embedding(301, embed_dim)
        self.lag_embedding = nn.Embedding(150, embed_dim)

        self.encoder = Encoder(
            d_model=embed_dim,n_layers=n_layers, n_head=n_head,\
            d_k=d_k, d_v=d_v, d_inner=d_inner, dropout=dropout)
        self.decoder = Decoder(
           d_model=embed_dim,n_layers=n_layers, n_head=n_head,\
            d_k=d_k, d_v=d_v, d_inner=d_inner, dropout=dropout)
        
        self.dropout = nn.Dropout(0.2)
        self.layer_normal = nn.LayerNorm(embed_dim) 

        self.ffn = PositionwiseFeedForward(d_in=embed_dim,d_hid=embed_dim,dropout=0.2)
        self.pred = nn.Linear(embed_dim, 1)
    
    def forward(self, x, question_ids,part_id, prior_question_elapsed_time,
               prior_lag_time):
        device = x.device        
        x = self.embedding(x)
        
        e = self.e_embedding(question_ids)
        e,x = self.pos_embedding(e,x)
        p = self.p_embedding(part_id)
        t = self.t_embedding(prior_question_elapsed_time)
        t_lag =self.lag_embedding(prior_lag_time)
        
        enc_input = e+p
        dec_input = x+t+t_lag
        
        
        enc_output, att_weight = self.encoder(enc_input, False )
        dec_output, *_ = self.decoder(dec_input,  enc_output, False )
        
        x = self.pred(dec_output)

        return x.squeeze(-1), att_weight

In [ ]:
dataset = RiiidDataset(group, n_skill)
dataloader = DataLoader(dataset, batch_size=224, shuffle=True,collate_fn=collate_fn)

item = dataset.__getitem__(5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:

model = SAINTPLUS( n_skill, max_seq=MAX_SEQ, embed_dim=128, d_inner=64,
            n_layers=5, n_head=8, d_k=64, d_v=64, dropout=0.1, n_position=MAX_SEQ)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.99, weight_decay=0.005)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCEWithLogitsLoss()
#model.load_state_dict(torch.load("SAINT+20epoch_CTS.pt", map_location=device))
criterion=criterion.to(device)
model=model.to(device)

In [ ]:
def train_epoch(model,train_iterator,optim,criterion, device="cuda"):
    train_loss = []
    num_corrects = 0
    num_total = 0
    labels = []
    outs = []
    tbar=tqdm(train_iterator)
    for batch in tbar:
        x,content_id,  part_id, prior_question_elapsed_time,\
        prior_lag_time,label,label_smoothing =batch
        x=x.to(device)
        content_id = content_id.to(device)
        part_id = part_id.to(device)
        prior_question_elapsed_time = prior_question_elapsed_time.to(device)
        prior_lag_time = prior_lag_time.to(device)

        label = label.to(device)
        label_smoothing = label_smoothing.to(device)
        '''
        print(content_id.size(),part_id.size(),\
              prior_question_elapsed_time.size(), mask.size(), label.size() )
        '''
        with torch.set_grad_enabled(mode=True):
            optimizer.zero_grad()
            output,atten_weight = model(x, content_id,part_id,
                           prior_question_elapsed_time,prior_lag_time)
            loss = criterion(output, label_smoothing)
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
            output = output[:,-1]
            label = label[:, -1] 
            pred = (torch.sigmoid(output) >= 0.5).long()
            num_corrects += (pred == label).sum().item()
            num_total += len(label)
            
            labels.extend(label.view(-1).data.cpu().numpy())
            outs.extend(output.view(-1).data.cpu().numpy())
            tbar.set_description('loss - {:.4f}'.format(loss))
    acc = num_corrects / num_total
    auc = roc_auc_score(labels, outs)
    loss = np.mean(train_loss)

    return loss, acc, auc

In [ ]:
device = 'cuda'
epochs = 10 #HDKIM 20
for epoch in range(epochs):
    loss, acc, auc = train_epoch(model, dataloader, optimizer, criterion, device)
    print("epoch - {} train_loss - {:.2f} acc - {:.3f} auc - {:.3f}".format(epoch, loss, acc, auc))
torch.save(model.state_dict(), "SAINT+labelsmoothing_notaug_10epo_5lay_128emb.pt")   